In [1]:
# Datset source
# https://archive.ics.uci.edu/ml/datasets/Appliances+energy+prediction

In [2]:
# Problem statement: Predict the appliances energy use based on various features

In [3]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [4]:
# Read the dataset

import pandas as pd
pd.options.display.max_columns = 1000
aep_df = pd.read_csv('energydata_complete.csv', sep=',')
print(aep_df.shape)
aep_df.head()

(19735, 29)


,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [5]:
# Check for NAN values in the entire dataframe

aep_df.isnull().sum().sum()

0

In [6]:
# To make this notebook's output identical at every run

np.random.seed(2)

In [7]:
# Split the dataframe into features and labels

X = aep_df.drop(['date', 'Appliances'], axis=1).values
y = aep_df.loc[:, 'Appliances'].values
print("X shape: ", X.shape, "y shape: ", y.shape)
print("Sample X values: ", X[:5], "\n", "Sample y values: ", y[:5])

X shape:  (19735, 27) y shape:  (19735,)
Sample X values:  [[ 30.          19.89        47.59666667  19.2         44.79
   19.79        44.73        19.          45.56666667  17.16666667
   55.2          7.02666667  84.25666667  17.2         41.62666667
   18.2         48.9         17.03333333  45.53         6.6
  733.5         92.           7.          63.           5.3
   13.27543316  13.27543316]
 [ 30.          19.89        46.69333333  19.2         44.7225
   19.79        44.79        19.          45.9925      17.16666667
   55.2          6.83333333  84.06333333  17.2         41.56
   18.2         48.86333333  17.06666667  45.56         6.48333333
  733.6         92.           6.66666667  59.16666667   5.2
   18.60619498  18.60619498]
 [ 30.          19.89        46.3         19.2         44.62666667
   19.79        44.93333333  18.92666667  45.89        17.16666667
   55.09         6.56        83.15666667  17.2         41.43333333
   18.2         48.73        17.          45.5   

In [8]:
# Split the dataset into train and test sets

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=2)

print(" X_train shape: ", X_train.shape,"\n", "y_train shape: ", y_train.shape,"\n",
        "X_test shape: ", X_test.shape,"\n", "y_test shape: ", y_test.shape,"\n")

 X_train shape:  (18748, 27) 
 y_train shape:  (18748,) 
 X_test shape:  (987, 27) 
 y_test shape:  (987,) 



In [9]:
# Scale the data

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [10]:
# Model 1
# Sklearn LinearSVR model with default parameters

from sklearn.svm import LinearSVR
lin_svr = LinearSVR(random_state=2)
lin_svr.fit(X_train_scaled, y_train)

LinearSVR(random_state=2)

In [11]:
# R^2 values for train and test sets

print("Train set R^2 score: ", lin_svr.score(X_train_scaled, y_train))
print("Test set R^2 score: ", lin_svr.score(X_test_scaled, y_test))

Train set R^2 score:  0.028301465312815077
Test set R^2 score:  0.03865841765464062


In [12]:
# Mean Squared Errors of train and test sets

from sklearn.metrics import mean_squared_error
print("Train set mse: ", mean_squared_error(y_train, lin_svr.predict(X_train_scaled)))
print("Test set mse: ", mean_squared_error(y_test, lin_svr.predict(X_test_scaled)))

Train set mse:  10289.28564720545
Test set mse:  8664.689117700937


In [13]:
# Mean Absolute Errors of train and test sets

from sklearn.metrics import mean_absolute_error
print("Train set mae: ", mean_absolute_error(y_train, lin_svr.predict(X_train_scaled)))
print("Test set mae: ", mean_absolute_error(y_test, lin_svr.predict(X_test_scaled)))

Train set mae:  44.19481061858397
Test set mae:  40.9262174168048


In [14]:
# LinearSVR with default hyperparameters is very poor at fitting the data, we will try to increase the R^2 score by using nonlinear kernels

In [15]:
# Model 2
# Sklearn SVR model with rbf kernel

from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

param_distributions = {"gamma": reciprocal(0.001, 1.0), "C": uniform(1, 10)}
rbf_rnd_search_cv = RandomizedSearchCV(SVR(), param_distributions, n_iter=30, n_jobs=6, verbose=5, cv=3, random_state=2)
rbf_rnd_search_cv.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


RandomizedSearchCV(cv=3, estimator=SVR(), n_iter=30, n_jobs=6,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f86eee6aad0>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f86eee6a310>},
                   random_state=2, verbose=5)

In [16]:
rbf_rnd_search_cv.best_estimator_

SVR(C=8.93637454441577, gamma=0.0549556737499024)

In [17]:
# R^2 values for train and test sets

print("Train set R^2 score: ", rbf_rnd_search_cv.best_estimator_.score(X_train_scaled, y_train))
print("Test set R^2 score: ", rbf_rnd_search_cv.best_estimator_.score(X_test_scaled, y_test))

Train set R^2 score:  0.13525206441821336
Test set R^2 score:  0.13859959472507088


In [18]:
# Mean Squared Errors of train and test sets

print("Train set mse: ", mean_squared_error(y_train, rbf_rnd_search_cv.best_estimator_.predict(X_train_scaled)))
print("Test set mse: ", mean_squared_error(y_test, rbf_rnd_search_cv.best_estimator_.predict(X_test_scaled)))

Train set mse:  9156.789070281558
Test set mse:  7763.907080103311


In [19]:
# Mean Absolute Errors of train and test sets

from sklearn.metrics import mean_absolute_error
print("Train set mae: ", mean_absolute_error(y_train, rbf_rnd_search_cv.best_estimator_.predict(X_train_scaled)))
print("Test set mae: ", mean_absolute_error(y_test, rbf_rnd_search_cv.best_estimator_.predict(X_test_scaled)))

Train set mae:  38.78134712925022
Test set mae:  36.509261084370316


In [20]:
# Model 3
# Sklearn SVR model with polynomial kernel

from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

poly_param_distributions = {"gamma": reciprocal(0.001, 1.0), "C": uniform(1, 10)}
poly_rnd_search_cv = RandomizedSearchCV(SVR(kernel='poly', degree=3, coef0=1), poly_param_distributions, n_iter=10, n_jobs=6, verbose=5, cv=3, random_state=2)
poly_rnd_search_cv.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3, estimator=SVR(coef0=1, kernel='poly'), n_jobs=6,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f86ef5bfe90>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f87b7fd2450>},
                   random_state=2, verbose=5)

In [21]:
poly_rnd_search_cv.best_estimator_

SVR(C=2.844398656469153, coef0=1, gamma=0.22698933025637727, kernel='poly')

In [22]:
# R^2 values for train and test sets

print("Train set R^2 score: ", poly_rnd_search_cv.best_estimator_.score(X_train_scaled, y_train))
print("Test set R^2 score: ", poly_rnd_search_cv.best_estimator_.score(X_test_scaled, y_test))

Train set R^2 score:  0.30789910771613627
Test set R^2 score:  0.2546722827162372


In [23]:
# Mean Squared Errors of train and test sets

print("Train set mse: ", mean_squared_error(y_train, poly_rnd_search_cv.best_estimator_.predict(X_train_scaled)))
print("Test set mse: ", mean_squared_error(y_test, poly_rnd_search_cv.best_estimator_.predict(X_test_scaled)))

Train set mse:  7328.63488333545
Test set mse:  6717.7297639763065


In [24]:
# Mean Absolute Errors of train and test sets

from sklearn.metrics import mean_absolute_error
print("Train set mae: ", mean_absolute_error(y_train, poly_rnd_search_cv.best_estimator_.predict(X_train_scaled)))
print("Test set mae: ", mean_absolute_error(y_test, poly_rnd_search_cv.best_estimator_.predict(X_test_scaled)))

Train set mae:  33.97802363970167
Test set mae:  34.90398855233448


In [26]:
# Model 3
# Sklearn SVR model with 5th order polynomial kernel

from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

poly_5_param_distributions = {"gamma": reciprocal(0.001, 1.0), "C": uniform(1, 10)}
poly_5_rnd_search_cv = RandomizedSearchCV(SVR(kernel='poly', degree=7, coef0=1), poly_5_param_distributions, n_iter=5, n_jobs=6, verbose=5, cv=3, random_state=2)
poly_5_rnd_search_cv.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


RandomizedSearchCV(cv=3, estimator=SVR(coef0=1, degree=5, kernel='poly'),
                   n_iter=5, n_jobs=6,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f87b7e93910>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f87b7e93810>},
                   random_state=2, verbose=5)

In [27]:
poly_5_rnd_search_cv.best_estimator_

SVR(C=3.046486340378425, coef0=1, degree=5, gamma=0.07207968815585897,
    kernel='poly')

In [28]:
# R^2 values for train and test sets

print("Train set R^2 score: ", poly_5_rnd_search_cv.best_estimator_.score(X_train_scaled, y_train))
print("Test set R^2 score: ", poly_5_rnd_search_cv.best_estimator_.score(X_test_scaled, y_test))

Train set R^2 score:  0.4258724771426182
Test set R^2 score:  0.3020110390896822


In [29]:
# Model 4
# Sklearn SVR model with 7th order polynomial kernel

from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

poly_7_param_distributions = {"gamma": reciprocal(0.001, 1.0), "C": uniform(1, 10)}
poly_7_rnd_search_cv = RandomizedSearchCV(SVR(kernel='poly', degree=7, coef0=1), poly_5_param_distributions, n_iter=5, n_jobs=6, verbose=5, cv=3, random_state=2)
poly_7_rnd_search_cv.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


RandomizedSearchCV(cv=3, estimator=SVR(coef0=1, degree=7, kernel='poly'),
                   n_iter=5, n_jobs=6,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f87b7e93910>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f87b7e93810>},
                   random_state=2, verbose=5)

In [30]:
poly_7_rnd_search_cv.best_estimator_

SVR(C=6.496624778787091, coef0=1, degree=7, gamma=0.02022866293198756,
    kernel='poly')

In [31]:
# R^2 values for train and test sets

print("Train set R^2 score: ", poly_7_rnd_search_cv.best_estimator_.score(X_train_scaled, y_train))
print("Test set R^2 score: ", poly_7_rnd_search_cv.best_estimator_.score(X_test_scaled, y_test))

Train set R^2 score:  0.2873208327379382
Test set R^2 score:  0.22623303870639755


In [32]:
# It turns out polynomial kernel model with degree 5 is a better model than linear svr, rbf model and polynomial kernel with other degrees with the specified set of parameters